In [2]:
import torch
from transformers import BigBirdForQuestionAnswering, BigBirdTokenizer

In [4]:
model_id = "google/bigbird-base-trivia-itc"
model = BigBirdForQuestionAnswering.from_pretrained(model_id, block_size=16, num_random_blocks=3)
tokenizer = BigBirdTokenizer.from_pretrained(model_id)

Downloading: 100%|██████████| 846k/846k [00:01<00:00, 842kB/s] 
Downloading: 100%|██████████| 943/943 [00:00<00:00, 212kB/s]


In [37]:
context = "🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between TensorFlow 2.0 and PyTorch. Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a question answering dataset is the SQuAD dataset"

In [38]:
def get_answer(question, context):
    encoding = tokenizer(question, context, return_tensors="pt", max_length=128, padding="max_length", truncation=True)
    input_ids = encoding.input_ids
    attention_mask = encoding.attention_mask

    with torch.no_grad():
        start_scores, end_scores = model(input_ids=input_ids, attention_mask=attention_mask).to_tuple()

    # Let's take the most likely token using `argmax` and retrieve the answer
    all_tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0].tolist())

    answer_tokens = all_tokens[torch.argmax(start_scores): torch.argmax(end_scores)+1]
    answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

    return answer

In [39]:
question = "How many pretrained models are available in 🤗 Transformers?"
get_answer(question, context)

'32'

In [3]:
import torch
from transformers import BigBirdPegasusForConditionalGeneration, BigBirdPegasusTokenizer

In [32]:
model_id = "vasudevgupta/bigbird-pegasus-large-bigpatent"
model = BigBirdPegasusForConditionalGeneration.from_pretrained(model_id, attention_type="original_full")
tokenizer = BigBirdPegasusTokenizer.from_pretrained(model_id)

In [33]:
article = """Punjab Kings new recruit Shahrukh Khan was spotted having a conversation with MS Dhoni after Chennai Super Kings cruised to a six-wicket win at the Wankhede Stadium on Friday. 
Khan made 47 off 36 balls and helped his side cross 100 after Deepak Chahar's four-for ran through Punjab's top-order. In the end, PBKS managed to gather only 106 for eight after reeling at 27 for five, and an attacking 66-run stand between Moeen Ali and Faf du Plessis helped CSK register their first victory of the season. 
After the win, skipper Dhoni, playing his 200th match for the Chennai-based outfit, was spotted passing tips to Shahrukh. The picture instantly went viral on social space after the official Twitter handle of IPL shared it after the match. Chennai Super Kings' official Twitter handle also shared a picture of a young Shahrukh and praised the 25-year-old's knock. Shahrukh had won the best all-rounder award at a Chennai Super Kings junior tournament when he was around 14.
"Lion forever! Well played Shahrukh, our Junior Super King, now a Punjab King," tweeted the franchise. 
Shahrukh, who has been entrusted with the task of finishing off matches for Punjab Kings, said that he has the ability to adapt and skills to bat anywhere.
"My role is playing down the order as a finisher but you don't expect to go out and blast in every game because there will be situations where the team bundles out and you need to take responsibility and see the team through," he said at the post-match press conference.
Known for his big-hitting prowess, the Tamil Nadu batsman struck jackpot with  INR 5.25 crore bid from Punjab Kings in the mini-auction earlier this year. 
"I'm branded as a complete finisher. I am a good batsman, I was batting up the order for Tamil Nadu for a couple of years.So, I've been batting at the top order. I've the skill in me to bat and see the team through in tough situations.
"I think I've really developed my game as a finisher in the last couple of years but still, my core game is still with me- See the ball and play normal cricketing shots," he added."""

In [34]:
inputs = tokenizer(article, max_length=512, padding="max_length", return_tensors="pt")
input_ids = inputs.input_ids
attention_mask = inputs.attention_mask
tokenizer.decode(input_ids.tolist()[0])

'Punjab Kings new recruit Shahrukh Khan was spotted having a conversation with MS Dhoni after Chennai Super Kings cruised to a six-wicket win at the Wankhede Stadium on Friday. Khan made 47 off 36 balls and helped his side cross 100 after Deepak Chahar\'s four-for ran through Punjab\'s top-order. In the end, PBKS managed to gather only 106 for eight after reeling at 27 for five, and an attacking 66-run stand between Moeen Ali and Faf du Plessis helped CSK register their first victory of the season. After the win, skipper Dhoni, playing his 200th match for the Chennai-based outfit, was spotted passing tips to Shahrukh. The picture instantly went viral on social space after the official Twitter handle of IPL shared it after the match. Chennai Super Kings\' official Twitter handle also shared a picture of a young Shahrukh and praised the 25-year-old\'s knock. Shahrukh had won the best all-rounder award at a Chennai Super Kings junior tournament when he was around 14. "Lion forever! Well p

In [35]:
len(input_ids[0])

512

In [36]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
o = model.generate(input_ids.to(device), attention_mask=attention_mask.to(device))

In [37]:
tokenizer.batch_decode(o.cpu())

['<s> The the “ The The The The The The The The The A “ the the the the the']